In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#importing packages we know we will need upfront
import seaborn as sns

from scipy.stats import iqr

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [3]:
# Uploading imputed data to df from CSV WorkingDataFrame
df = pd.read_csv('LMPDStopsDatFrame.csv', index_col=0)
df.head()

,ID,TYPE_OF_STOP,CITATION_CONTROL_NUMBER,ACTIVITY RESULTS,OFFICER_GENDER,OFFICER_RACE,OFFICER_AGE_RANGE,ACTIVITY_DATE,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,ACTIVITY_BEAT,DRIVER_GENDER,DRIVER_RACE,DRIVER_AGE_RANGE,NUMBER OF PASSENGERS,WAS_VEHCILE_SEARCHED,REASON_FOR_SEARCH
0,1,NaN,NaN,WARNING,NaN,NaN,NaN,2015-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,2,NaN,NaN,WARNING,NaN,NaN,NaN,2015-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,3,NaN,NaN,WARNING,NaN,NaN,NaN,2016-09-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,4,NaN,NaN,WARNING,F,WHITE,21 - 30,2016-01-22,14:42:00,SHAWNEE PARK,2ND DIVISION,BEAT 1,M,BLACK,31 - 40,3,NO,NaN
4,5,NaN,NaN,WARNING,F,WHITE,21 - 30,2015-09-14,21:10:00,RIVERPARK/34TH,2ND DIVISION,BEAT 1,M,BLACK,20 - 25,0,YES,PROBABLE CAUSE


# Data Imputation & Cleaning #

In [4]:
#fill missing values for each column using its own most frequent value
df = df.fillna(df.mode().iloc[0])
df.isnull().sum()

ID                         0
TYPE_OF_STOP               0
CITATION_CONTROL_NUMBER    0
ACTIVITY RESULTS           0
OFFICER_GENDER             0
OFFICER_RACE               0
OFFICER_AGE_RANGE          0
ACTIVITY_DATE              0
ACTIVITY_TIME              0
ACTIVITY_LOCATION          0
ACTIVITY_DIVISION          0
ACTIVITY_BEAT              0
DRIVER_GENDER              0
DRIVER_RACE                0
DRIVER_AGE_RANGE           0
NUMBER OF PASSENGERS       0
WAS_VEHCILE_SEARCHED       0
REASON_FOR_SEARCH          0
dtype: int64

In [5]:
# Creating a copy of the dataset, to recode values to be numerical for data analysis. 
df2 = df.copy(deep=True)
df2.head()

,ID,TYPE_OF_STOP,CITATION_CONTROL_NUMBER,ACTIVITY RESULTS,OFFICER_GENDER,OFFICER_RACE,OFFICER_AGE_RANGE,ACTIVITY_DATE,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,ACTIVITY_BEAT,DRIVER_GENDER,DRIVER_RACE,DRIVER_AGE_RANGE,NUMBER OF PASSENGERS,WAS_VEHCILE_SEARCHED,REASON_FOR_SEARCH
0,1,TRAFFIC VIOLATION,0,WARNING,M,WHITE,40 - 50,2015-01-24,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
1,2,TRAFFIC VIOLATION,0,WARNING,M,WHITE,40 - 50,2015-01-30,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
2,3,TRAFFIC VIOLATION,0,WARNING,M,WHITE,40 - 50,2016-09-20,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
3,4,TRAFFIC VIOLATION,0,WARNING,F,WHITE,21 - 30,2016-01-22,14:42:00,SHAWNEE PARK,2ND DIVISION,BEAT 1,M,BLACK,31 - 40,3,NO,CONSENT
4,5,TRAFFIC VIOLATION,0,WARNING,F,WHITE,21 - 30,2015-09-14,21:10:00,RIVERPARK/34TH,2ND DIVISION,BEAT 1,M,BLACK,20 - 25,0,YES,PROBABLE CAUSE


In [6]:
print(df2.nunique())

ID                         111582
TYPE_OF_STOP                    3
CITATION_CONTROL_NUMBER     82661
ACTIVITY RESULTS                2
OFFICER_GENDER                  2
OFFICER_RACE                    5
OFFICER_AGE_RANGE               5
ACTIVITY_DATE                1125
ACTIVITY_TIME                1440
ACTIVITY_LOCATION           47718
ACTIVITY_DIVISION              10
ACTIVITY_BEAT                   8
DRIVER_GENDER                   2
DRIVER_RACE                    10
DRIVER_AGE_RANGE                8
NUMBER OF PASSENGERS           26
WAS_VEHCILE_SEARCHED            2
REASON_FOR_SEARCH             296
dtype: int64


In [7]:
df2['ACTIVITY RESULTS']=df2['ACTIVITY RESULTS'].replace({'WARNING': 0, 'CITATION ISSUED': 1})
df2.head()

,ID,TYPE_OF_STOP,CITATION_CONTROL_NUMBER,ACTIVITY RESULTS,OFFICER_GENDER,OFFICER_RACE,OFFICER_AGE_RANGE,ACTIVITY_DATE,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,ACTIVITY_BEAT,DRIVER_GENDER,DRIVER_RACE,DRIVER_AGE_RANGE,NUMBER OF PASSENGERS,WAS_VEHCILE_SEARCHED,REASON_FOR_SEARCH
0,1,TRAFFIC VIOLATION,0,0,M,WHITE,40 - 50,2015-01-24,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
1,2,TRAFFIC VIOLATION,0,0,M,WHITE,40 - 50,2015-01-30,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
2,3,TRAFFIC VIOLATION,0,0,M,WHITE,40 - 50,2016-09-20,22:00:00,DIXIE HWY ...,6TH DIVISION,BEAT 2,M,WHITE,31 - 40,0,NO,CONSENT
3,4,TRAFFIC VIOLATION,0,0,F,WHITE,21 - 30,2016-01-22,14:42:00,SHAWNEE PARK,2ND DIVISION,BEAT 1,M,BLACK,31 - 40,3,NO,CONSENT
4,5,TRAFFIC VIOLATION,0,0,F,WHITE,21 - 30,2015-09-14,21:10:00,RIVERPARK/34TH,2ND DIVISION,BEAT 1,M,BLACK,20 - 25,0,YES,PROBABLE CAUSE


In [8]:
X1 = df2.filter(['TYPE_OF_STOP', 'OFFICER_GENDER', 'DRIVER_GENDER', 'ACTIVITY_BEAT', 'OFFICER_AGE_RANGE', 'DRIVER_AGE_RANGE',
       'WAS_VEHCILE_SEARCHED', 'NUMBER OF PASSENGERS', 'ACTIVITY_DIVISION'])
X2 = df2.filter(['ID', 'CITATION_CONTROL_NUMBER', 'ACTIVITY_DATE', 'ACTIVITY_TIME', 'ACTIVITY_LOCATION', 
        'ACTIVITY_DIVISION', 'REASON_FOR_SEARCH', 'OFFICER_AGE_RANGE', 'DRIVER_AGE_RANGE'])

y = df2.iloc[:, 3]


In [9]:
X1 = pd.get_dummies(X1)
X1.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_DIVISION_,ACTIVITY_DIVISION_1ST DIVISION,ACTIVITY_DIVISION_2ND DIVISION,ACTIVITY_DIVISION_3RD DIVISION,ACTIVITY_DIVISION_4TH DIVISION,ACTIVITY_DIVISION_5TH DIVISION,ACTIVITY_DIVISION_6TH DIVISION,ACTIVITY_DIVISION_7TH DIVISION,ACTIVITY_DIVISION_8TH DIVISION,ACTIVITY_DIVISION_NON LMPD
0,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3,0,0,1,1,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,1,1,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0


In [10]:
df_concat = pd.concat([X1, X2], axis=1, sort=False)
df_concat.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_DIVISION_NON LMPD,ID,CITATION_CONTROL_NUMBER,ACTIVITY_DATE,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,REASON_FOR_SEARCH,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE
0,0,0,0,1,0,1,0,1,0,0,...,0,1,0,2015-01-24,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40
1,0,0,0,1,0,1,0,1,0,0,...,0,2,0,2015-01-30,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40
2,0,0,0,1,0,1,0,1,0,0,...,0,3,0,2016-09-20,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40
3,3,0,0,1,1,0,0,1,0,1,...,0,4,0,2016-01-22,14:42:00,SHAWNEE PARK,2ND DIVISION,CONSENT,21 - 30,31 - 40
4,0,0,0,1,1,0,0,1,0,1,...,0,5,0,2015-09-14,21:10:00,RIVERPARK/34TH,2ND DIVISION,PROBABLE CAUSE,21 - 30,20 - 25


In [11]:
#replacing year with 2025 to 2015.. assuming a typo 
df_concat['ACTIVITY_DATE'] = df_concat['ACTIVITY_DATE'].str.replace("2025","2015")

In [12]:
#converting time column data type to Datetime64
df_concat['ACTIVITY_DATE']=df_concat['ACTIVITY_DATE'].astype('Datetime64',copy=True) 

In [13]:
#Adding new column of just the year of the activity date
df_concat['ACTIVITY_MONTH_YEAR'] = df_concat['ACTIVITY_DATE'].dt.strftime('%m/%y')
df_concat['ACTIVITY_MONTH'] = df_concat['ACTIVITY_DATE'].dt.strftime('%m')
df_concat['ACTIVITY_DAY'] = df_concat['ACTIVITY_DATE'].dt.strftime('%d') 
df_concat['ACTIVITY_YEAR'] = df_concat['ACTIVITY_DATE'].dt.strftime('%y')
df_concat.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,REASON_FOR_SEARCH,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE,ACTIVITY_MONTH_YEAR,ACTIVITY_MONTH,ACTIVITY_DAY,ACTIVITY_YEAR
0,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,24,15
1,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,30,15
2,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,09/16,09,20,16
3,3,0,0,1,1,0,0,1,0,1,...,14:42:00,SHAWNEE PARK,2ND DIVISION,CONSENT,21 - 30,31 - 40,01/16,01,22,16
4,0,0,0,1,1,0,0,1,0,1,...,21:10:00,RIVERPARK/34TH,2ND DIVISION,PROBABLE CAUSE,21 - 30,20 - 25,09/15,09,14,15


In [14]:
#Adding new column of just the year of the activity date
df_concat['ACTIVITY_MONTH_YEAR'] = df_concat['ACTIVITY_DATE'].dt.strftime('%m/%y')
df_concat['ACTIVITY_MONTH'] = df_concat['ACTIVITY_DATE'].dt.strftime('%m')
df_concat['ACTIVITY_DAY'] = df_concat['ACTIVITY_DATE'].dt.strftime('%d') 
df_concat['ACTIVITY_YEAR'] = df_concat['ACTIVITY_DATE'].dt.strftime('%y')
df_concat.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,REASON_FOR_SEARCH,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE,ACTIVITY_MONTH_YEAR,ACTIVITY_MONTH,ACTIVITY_DAY,ACTIVITY_YEAR
0,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,24,15
1,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,30,15
2,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,09/16,09,20,16
3,3,0,0,1,1,0,0,1,0,1,...,14:42:00,SHAWNEE PARK,2ND DIVISION,CONSENT,21 - 30,31 - 40,01/16,01,22,16
4,0,0,0,1,1,0,0,1,0,1,...,21:10:00,RIVERPARK/34TH,2ND DIVISION,PROBABLE CAUSE,21 - 30,20 - 25,09/15,09,14,15


#Changing column Activity_Time from 24 hour to 12 hour 
import datetime
times = df_concat['ACTIVITY_TIME']
df_concat['ACTIVITY_TIME']=[datetime.datetime.strptime(time, "%H:%M:%S").strftime("%I:%M %p") for time in times]

In [15]:
#Replacing 'over 60' with age range 61-70, and 'UNDER 16' with age range 15-10
df_concat = df_concat.replace({'OFFICER_AGE_RANGE': {'OVER 60': '61 - 70'}})
df_concat = df_concat.replace({'DRIVER_AGE_RANGE': {'OVER 60': '61 - 70', 'UNDER 16': '15 - 10'}})
df_concat.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_TIME,ACTIVITY_LOCATION,ACTIVITY_DIVISION,REASON_FOR_SEARCH,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE,ACTIVITY_MONTH_YEAR,ACTIVITY_MONTH,ACTIVITY_DAY,ACTIVITY_YEAR
0,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,24,15
1,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,30,15
2,0,0,0,1,0,1,0,1,0,0,...,22:00:00,DIXIE HWY ...,6TH DIVISION,CONSENT,40 - 50,31 - 40,09/16,09,20,16
3,3,0,0,1,1,0,0,1,0,1,...,14:42:00,SHAWNEE PARK,2ND DIVISION,CONSENT,21 - 30,31 - 40,01/16,01,22,16
4,0,0,0,1,1,0,0,1,0,1,...,21:10:00,RIVERPARK/34TH,2ND DIVISION,PROBABLE CAUSE,21 - 30,20 - 25,09/15,09,14,15


In [16]:
#using split_mean() function to split age ranges to calculate mean, added columns'OFFICER_AGE_MEAN' & 'DRIVER_AGE_MEAN'
def split_mean(x):
    split_list = x.split('-')
    mean = (float(split_list[0])+float(split_list[1]))/2
    return mean

df_concat['OFFICER_AGE_MEAN'] = df_concat['OFFICER_AGE_RANGE'].apply(lambda x: split_mean(x))
df_concat['DRIVER_AGE_MEAN'] = df_concat['DRIVER_AGE_RANGE'].apply(lambda x: split_mean(x))

In [17]:
df_concat.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,ACTIVITY_DIVISION,REASON_FOR_SEARCH,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE,ACTIVITY_MONTH_YEAR,ACTIVITY_MONTH,ACTIVITY_DAY,ACTIVITY_YEAR,OFFICER_AGE_MEAN,DRIVER_AGE_MEAN
0,0,0,0,1,0,1,0,1,0,0,...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,24,15,45.0,35.5
1,0,0,0,1,0,1,0,1,0,0,...,6TH DIVISION,CONSENT,40 - 50,31 - 40,01/15,01,30,15,45.0,35.5
2,0,0,0,1,0,1,0,1,0,0,...,6TH DIVISION,CONSENT,40 - 50,31 - 40,09/16,09,20,16,45.0,35.5
3,3,0,0,1,1,0,0,1,0,1,...,2ND DIVISION,CONSENT,21 - 30,31 - 40,01/16,01,22,16,25.5,35.5
4,0,0,0,1,1,0,0,1,0,1,...,2ND DIVISION,PROBABLE CAUSE,21 - 30,20 - 25,09/15,09,14,15,25.5,22.5


In [18]:
df_concat['TIME_AM/PM'] = df_concat['ACTIVITY_TIME']

In [19]:
#Changing column Activity_Time from 24 hour to 12 hour 
import datetime
times = df_concat['TIME_AM/PM']
df_concat['TIME_AM/PM']=[datetime.datetime.strptime(time, "%H:%M:%S").strftime("%I:%M %p") for time in times]

In [20]:
df_concat['ACTIVITY_TIME'] = pd.to_datetime(df_concat['ACTIVITY_TIME'], errors='coerce')

In [42]:
df = df_concat.assign(Session=pd.cut(df_concat['ACTIVITY_TIME'].dt.hour,[-1,6,12,18,24],labels=['Night','Morning','Afternoon','Evening']))


In [46]:
df.head()

,NUMBER OF PASSENGERS,TYPE_OF_STOP_COMPLAINT/CRIMINAL VIOLATION,TYPE_OF_STOP_COMPLIANCE STOP (KVE ONLY),TYPE_OF_STOP_TRAFFIC VIOLATION,OFFICER_GENDER_F,OFFICER_GENDER_M,DRIVER_GENDER_F,DRIVER_GENDER_M,ACTIVITY_BEAT_,ACTIVITY_BEAT_BEAT 1,...,OFFICER_AGE_RANGE,DRIVER_AGE_RANGE,ACTIVITY_MONTH_YEAR,ACTIVITY_MONTH,ACTIVITY_DAY,ACTIVITY_YEAR,OFFICER_AGE_MEAN,DRIVER_AGE_MEAN,TIME_AM/PM,Session
0,0,0,0,1,0,1,0,1,0,0,...,40 - 50,31 - 40,01/15,01,24,15,45.0,35.5,10:00 PM,Evening
1,0,0,0,1,0,1,0,1,0,0,...,40 - 50,31 - 40,01/15,01,30,15,45.0,35.5,10:00 PM,Evening
2,0,0,0,1,0,1,0,1,0,0,...,40 - 50,31 - 40,09/16,09,20,16,45.0,35.5,10:00 PM,Evening
3,3,0,0,1,1,0,0,1,0,1,...,21 - 30,31 - 40,01/16,01,22,16,25.5,35.5,02:42 PM,Afternoon
4,0,0,0,1,1,0,0,1,0,1,...,21 - 30,20 - 25,09/15,09,14,15,25.5,22.5,09:10 PM,Evening


In [ ]:
#df_concat.to_csv('ImputedData', ',')